This is my first sharing notebook in kaggle! Thank you for reading. And I learned a lot on this competition.

### transformers==4.24.0 for EsmTokenizer and EsmMaskedLM

In [1]:
!pip install transformers==4.24.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 37.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.20.1
    Uninstalling transformers-4.20.1:
      Successfully uninstalled transformers-4.20.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.10.1 requires transformers<4.21,>=4.1, but you have transformers 4.24.0 which is incompatible.


In [2]:
import pandas as pd
import numpy as np

import transformers
from transformers import EsmTokenizer, EsmForMaskedLM
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Load esm2_t33_650M_UR50D

In [3]:
# If we use large scale model(3B, 15B), we can get higher score than 650M.
tokenizer = EsmTokenizer.from_pretrained("facebook/esm2_t33_650M_UR50D")
model = EsmForMaskedLM.from_pretrained("facebook/esm2_t33_650M_UR50D").to(device)

Downloading:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.61G [00:00<?, ?B/s]

### Load test sequence

In [4]:
seqs = list(pd.read_csv('../input/novozymes-enzyme-stability-prediction/test.csv')['protein_sequence'])

### Simply calculate pseudo-perplexity for each sequence

In [5]:
def score(model, tokenizer, sentence):
    tensor_input = tokenizer.encode(sentence, return_tensors='pt')
    repeat_input = tensor_input.repeat(tensor_input.size(-1)-2, 1)
    
    # mask one by one except [CLS] and [SEP]
    mask = torch.ones(tensor_input.size(-1) -1).diag(1)[:-2]
    masked_input = repeat_input.masked_fill(mask == 1, tokenizer.mask_token_id)
    
    labels = repeat_input.masked_fill(masked_input != tokenizer.mask_token_id, -100)
    with torch.no_grad():
        loss = model(masked_input.to(device), labels=labels.to(device)).loss
    return np.exp(loss.item())

In [6]:
ppl = []
for i in seqs:
    num = score(model=model, tokenizer=tokenizer, sentence = i)
    ppl.append(num)

In [7]:
from scipy import stats

test_df = pd.read_csv('../input/novozymes-enzyme-stability-prediction/test.csv')
test_df['score'] = ppl

# Lower Pseudo-Perplexity is better
test_df['rank'] = len(test_df['score']) - stats.rankdata(test_df['score']) + 1

In [8]:
submit_df = pd.DataFrame({
    'seq_id': test_df.seq_id.values,
    'tm': test_df['rank'].values,
})
submit_df.tm = submit_df.tm.fillna(0)
submit_df.to_csv('ESM_perplexity.csv', index=False)
submit_df.head()

,seq_id,tm
0,31390,2360.0
1,31391,2134.0
2,31392,2310.0
3,31393,1781.0
4,31394,980.0
